In [50]:
# install.packages('StatMatch')
# install.packages('reshape')
# install.packages('ade4')
# install.packagesckages('stringi')
# install.packages('Hmisc')""
# install.packages("tidyr")


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [52]:
library(StatMatch)
library(reshape)
library(ade4)
library(Hmisc)
library(tidyr)

In [132]:
# Read in data
recs = read.csv("./data/recs2015_public_v4.csv")[1:2000,]
pums<- read.csv("./data/ss15hpa.csv")[1:10000,]

In [210]:
# reformating common variables in recs
recs$BTUOTHER <- recs$TOTALBTU - recs$BTUEL - recs$BTUNG

recs$x_hhtype <- factor(recs$TYPEHUQ, labels = c(" mobile home", " single family detached", " single family attached", " apartment in building with 2-4 units", " apartment in building with 5+ units"))
recs$x_tenure <- factor(recs$KOWNRENT, labels = c(" owned by someone in the household", " rented", " occupied without payment of rent"))

recs$x_yearbuilt <- factor(recs$YEARMADERANGE,
                        labels = c(" 1950 or earlier", " 1950-1959", " 1960-1969", " 1970-1979",
                                   " 1980-1989"," 1990-1999", " 2000-2009", " 2010-2015"), order = TRUE)

recs$x_mvin <- cut(recs$OCCUPYYRANGE, breaks = c(-Inf, 4, 5, 6,7,Inf), labels = c(" 30 years or more", " 20-29 years", " 10-19 years", " 5-9 years", " 4 or less years"), ordered = TRUE)

recs$BEDROOMS[recs$BEDROOMS<0] <- NA
recs$x_bedrooms <- recs$BEDROOMS

recs$x_totrooms <- recs$TOTROOMS

recs$x_heatfuel <- cut(recs$FUELHEAT, breaks = c(-Inf, 0, 1, 2, 4,5,7, Inf), labels = c(" No fuel used", " natural gas", " propange/LPG", " fuel oil or Kerosene", " electricity", " wood", " other fuel"))
recs$x_heatfuel <- recs$FUELHEAT
recs$x_heatfuel[recs$x_heatfuel == 2] <- 8
recs$x_heatfuel[recs$x_heatfuel == 4] <- 8
recs$x_heatfuel[recs$x_heatfuel == 7] <- 8
recs$x_heatfuel[recs$x_heatfuel == 0] <- 8
recs$x_heatfuel <- cut(recs$x_heatfuel, breaks = c(-Inf, 1, 5,Inf), labels = c( " natural gas",  " electricity", " other fuel"))

recs$x_hhsize <- recs$NHSLDMEM
recs$x_hhincome <- factor(recs$MONEYPY, labels = c("Less than $20,000",
"$20,000 - $39,999",
"$40,000 - $59,999",
"$60,000 to $79,999",
"$80,000 to $99,999",
"$100,000 to $119,999",
"$120,000 to $139,999",
"$140,000 or more"),ordered = TRUE)
recs$x_yearel <- recs$DOLLAREL

recs$HDD65[recs$HDD65 ==0 ] = mean(recs$HDD65)

recs$x_yearng <- recs$DOLLARNG/recs$HDD65*374 #https://www.eia.gov/totalenergy/data/annual/showtext.php?t=ptb0107
recs$x_yearother <- recs$TOTALDOL - recs$DOLLAREL - recs$DOLLARNG


In [213]:
# reformating common variables in pums
pums<- pums[,1:82] #get rid of the weights
pums$x_hhtype <- cut(pums$BLD, breaks= c(-Inf, 1,2,3,5,Inf), labels = c(" mobile home", " single family detached", " single family attached", " apartment in building with 2-4 units", " apartment in building with 5+ units"))
pums$x_tenure <- cut(pums$TEN, breaks = c(-Inf, 2, 3, Inf), labels = c(" owned by someone in the household", " rented", " occupied without payment of rent"))

pums$x_yearbuilt <- cut(replace_na(pums$YBL, 20), 
                        breaks = c(-Inf, 2, 3, 4, 5, 6, 7, 13, 19, Inf),
                        labels = c(" 1950 or earlier", " 1950-1959", " 1960-1969", " 1970-1979",
                                   " 1980-1989"," 1990-1999", " 2000-2009", " 2010-2015", 'UNK'),
                        ordered = TRUE)


pums$x_mvin <- cut(pums$MV, breaks = c(-Inf, 3, 4, 5, 6,Inf), labels = c(" 4 or less years"," 5-9 years" , " 10-19 years",  " 20-29 years"," 30 years or more" ), ordered = TRUE)
pums$x_bedrooms <- pums$BDSP
pums$x_totrooms <- pums$RMSP
pums$x_heatfuel <- pums$HFL
pums$x_heatfuel[pums$x_heatfuel==5] <- 8
pums$x_heatfuel[pums$x_heatfuel==2] <- 8
pums$x_heatfuel[pums$x_heatfuel==4] <- 8
pums$x_heatfuel[pums$x_heatfuel==6] <- 8
pums$x_heatfuel[pums$x_heatfuel==9] <- 8
pums$x_heatfuel <- cut(pums$x_heatfuel, breaks = c(-Inf, 1,3, Inf), labels = c( " natural gas", " electricity",  " other fuel"))
pums$x_hhsize <- pums$NP
pums$x_hhincome <- addNA(cut(pums$HINCP,
    breaks = c(-Inf, 20000, 40000, 60000, 80000, 100000, 120000, 140000, Inf),
    labels = c("Less than $20,000",
                "$20,000 to $39,999",
                "$40,000 to $59,999",
                "$60,000 to $79,999",
                "$80,000 to $99,999",
                "$100,000 to $119,999",
                "$120,000 to $139,999",
                "$140,000 or more"),ordered = TRUE))

pums$x_yearel <- pums$ELEP*12
pums$x_yearel[pums$x_yearel==24] <- 0
pums$x_yearel[pums$x_yearel==12] <- 0


pums$x_yearng <- pums$GASP
pums$x_yearng[pums$x_yearng==1] <- 0
pums$x_yearng[pums$x_yearng==2] <- 0
pums$x_yearng[pums$x_yearng==3] <- 0

pums$x_yearother <- pums$FULP
pums$x_yearother[pums$x_yearother==1] <- 0
pums$x_yearother[pums$x_yearother==2] <- 0

spearman2(TOTALBTU~x_hhtype+x_tenure+x_yearbuilt+x_mvin+x_bedrooms+x_totrooms+x_heatfuel+x_hhsize+ x_hhincome+ x_yearel+ x_yearng+ x_yearother, p =2, data = recs)
spearman2(BTUEL~x_hhtype+x_tenure+x_yearbuilt+x_mvin+x_bedrooms+x_totrooms+x_heatfuel+x_hhsize+ x_hhincome+ x_yearel+ x_yearng+ x_yearother, p =2, data = recs)
spearman2(BTUNG~x_hhtype+x_tenure+x_yearbuilt+x_mvin+x_bedrooms+x_totrooms+x_heatfuel+x_hhsize+ x_hhincome+ x_yearel+ x_yearng+ x_yearother, p =2, data = recs)
spearman2(BTUOTHER~x_hhtype+x_tenure+x_yearbuilt+x_mvin+x_bedrooms+x_totrooms+x_heatfuel+x_hhsize+ x_hhincome+ x_yearel+ x_yearng+ x_yearother, p =2, data = recs)


Spearman rho^2    Response variable:TOTALBTU

             rho2      F df1  df2 P Adjusted rho2    n
x_hhtype    0.347 264.54   4 1995 0         0.345 2000
x_tenure    0.210 265.35   2 1997 0         0.209 2000
x_yearbuilt 0.029   8.45   7 1992 0         0.025 2000
x_mvin      0.092  50.83   4 1995 0         0.091 2000
x_bedrooms  0.320 469.45   2 1997 0         0.319 2000
x_totrooms  0.367 578.57   2 1997 0         0.366 2000
x_heatfuel  0.120 136.37   2 1997 0         0.119 2000
x_hhsize    0.098 108.10   2 1997 0         0.097 2000
x_hhincome  0.139  45.77   7 1992 0         0.136 2000
x_yearel    0.216 274.64   2 1997 0         0.215 2000
x_yearng    0.153 180.04   2 1997 0         0.152 2000
x_yearother 0.190 233.80   2 1997 0         0.189 2000


Spearman rho^2    Response variable:BTUEL

             rho2       F df1  df2      P Adjusted rho2    n
x_hhtype    0.188  115.18   4 1995 0.0000         0.186 2000
x_tenure    0.101  112.55   2 1997 0.0000         0.100 2000
x_yearbuilt 0.046   13.60   7 1992 0.0000         0.042 2000
x_mvin      0.045   23.46   4 1995 0.0000         0.043 2000
x_bedrooms  0.187  230.05   2 1997 0.0000         0.186 2000
x_totrooms  0.192  237.03   2 1997 0.0000         0.191 2000
x_heatfuel  0.107  119.75   2 1997 0.0000         0.106 2000
x_hhsize    0.114  128.75   2 1997 0.0000         0.113 2000
x_hhincome  0.057   17.06   7 1992 0.0000         0.053 2000
x_yearel    0.834 5029.95   2 1997 0.0000         0.834 2000
x_yearng    0.174  209.71   2 1997 0.0000         0.173 2000
x_yearother 0.002    2.01   2 1997 0.1344         0.001 2000


Spearman rho^2    Response variable:BTUNG

             rho2       F df1  df2     P Adjusted rho2    n
x_hhtype    0.066   35.29   4 1995 0e+00         0.064 2000
x_tenure    0.030   31.15   2 1997 0e+00         0.029 2000
x_yearbuilt 0.062   18.81   7 1992 0e+00         0.059 2000
x_mvin      0.011    5.70   4 1995 1e-04         0.009 2000
x_bedrooms  0.057   60.25   2 1997 0e+00         0.056 2000
x_totrooms  0.063   67.20   2 1997 0e+00         0.062 2000
x_heatfuel  0.708 2417.07   2 1997 0e+00         0.707 2000
x_hhsize    0.012   11.97   2 1997 0e+00         0.011 2000
x_hhincome  0.041   12.28   7 1992 0e+00         0.038 2000
x_yearel    0.037   38.56   2 1997 0e+00         0.036 2000
x_yearng    0.850 5667.82   2 1997 0e+00         0.850 2000
x_yearother 0.100  110.96   2 1997 0e+00         0.099 2000


Spearman rho^2    Response variable:BTUOTHER

             rho2      F df1  df2      P Adjusted rho2    n
x_hhtype    0.015   7.36   4 1995 0.0000         0.013 2000
x_tenure    0.007   6.75   2 1997 0.0012         0.006 2000
x_yearbuilt 0.007   2.09   7 1992 0.0416         0.004 2000
x_mvin      0.011   5.78   4 1995 0.0001         0.009 2000
x_bedrooms  0.002   1.97   2 1997 0.1398         0.001 2000
x_totrooms  0.003   2.97   2 1997 0.0513         0.002 2000
x_heatfuel  0.136 157.66   2 1997 0.0000         0.135 2000
x_hhsize    0.002   1.52   2 1997 0.2199         0.001 2000
x_hhincome  0.003   0.92   7 1992 0.4907         0.000 2000
x_yearel    0.004   3.63   2 1997 0.0267         0.003 2000
x_yearng    0.065  69.32   2 1997 0.0000         0.064 2000
x_yearother 0.316 462.29   2 1997 0.0000         0.316 2000

In [214]:
# drop NA values and standardize common variables
pums <- pums[complete.cases(pums[,c("x_hhtype", "x_heatfuel", "x_totrooms", "x_yearel", "x_yearng", "x_yearother")]),]

merge_totrooms <- c(pums$x_totrooms, recs$x_totrooms)
sd <- sd(merge_totrooms, na.rm = TRUE)
mean <- mean(merge_totrooms, na.rm = TRUE)
pums$x_totrooms_std <- (pums$x_totrooms - mean)/sd
recs$x_totrooms_std <- (recs$x_totrooms - mean)/sd

merge_yearel <- c(pums$x_yearel, recs$x_yearel)
sd <- sd(merge_yearel, na.rm = TRUE)
mean <- mean(merge_yearel, na.rm = TRUE)
pums$x_yearel_std <- (pums$x_yearel - mean)/sd
recs$x_yearel_std <- (recs$x_yearel - mean)/sd

merge_yearng <- c(pums$x_yearng, recs$x_yearng)
sd <- sd(merge_yearng, na.rm = TRUE)
mean <- mean(merge_yearng, na.rm = TRUE)
pums$x_yearng_std <- (pums$x_yearng - mean)/sd
recs$x_yearng_std <- (recs$x_yearng - mean)/sd

merge_yearother <- c(pums$x_yearother, recs$x_yearother)
sd <- sd(merge_yearother, na.rm = TRUE)
mean <- mean(merge_yearother, na.rm = TRUE)
pums$x_yearother_std <- (pums$x_yearother - mean)/sd
recs$x_yearother_std <- (recs$x_yearother - mean)/sd

merge_hhsize <- c(pums$x_hhsize, recs$x_hhsize)
sd <- sd(merge_hhsize, na.rm = TRUE)
mean <- mean(merge_hhsize, na.rm = TRUE)
pums$x_hhsize_std <- (pums$x_hhsize - mean)/sd
recs$x_hhsize_std <- (recs$x_hhsize - mean)/sd


In [217]:
# statmatch electricity BTU
group.v <- c("x_hhtype")
X.mtc <- c("x_totrooms_std", "x_yearel_std")
X.join <- c("SERIALNO","x_totrooms","x_yearel","x_yearng", "x_hhtype", "x_heatfuel")

out.el <- NND.hotdeck(data.rec = recs, data.don = pums, match.vars = X.mtc, don.class = group.v, constrained = TRUE )
elbtu <- create.fused(data.rec = recs, data.don = pums, mtc.ids = out.el$mtc.ids, z.vars=X.join)
elbtu$dist <- out.el$dist.rd
summary(elbtu$dist)
elbtu<- elbtu[elbtu$dist<1,] #cut off distance above 1

elbtu <- elbtu[,c("SERIALNO","BTUEL")]
pums_el_final <- merge(pums, elbtu, by="SERIALNO", all.x = TRUE)
el_training <- pums_el_final[complete.cases(pums_el_final[,"BTUEL"]),]
el_imputing <- pums_el_final[is.na(pums_el_final$"BTUEL"),]


All the categorical matching variables in rec and don 
 data.frames, if present are recoded into dummies


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.00000 0.01500 0.02914 0.03916 0.04328 0.87124 

In [218]:
# prepare electric outputs for machine learning
el_training <- el_training[,-83:-99]
el_imputing <- el_imputing[,-83:-100]
el_training[is.na(el_training)] <- 0
el_imputing[is.na(el_imputing)] <- 0

write.csv(el_training, "output/el_training.csv", row.names =FALSE)
write.csv(el_imputing, "output/el_imputing.csv", row.names =FALSE)


In [219]:
# statmatch natural gas BTU
group.v <- c("x_heatfuel")
X.mtc <- c("x_yearng_std")
X.join <- c("SERIALNO","x_yearng", "x_heatfuel")


In [220]:
out.ng <- NND.hotdeck(data.rec = recs, data.don = pums, match.vars = X.mtc, don.class = group.v, constrained = TRUE )
ngbtu <- create.fused(data.rec = recs, data.don = pums, mtc.ids = out.ng$mtc.ids, z.vars=X.join)
ngbtu$dist <- out.ng$dist.rd
ngbtu<- ngbtu[ngbtu$dist<1,]

ngbtu <- ngbtu[,c("SERIALNO","BTUNG")]
pums_ng_final <- merge(pums, ngbtu, by="SERIALNO", all.x = TRUE)
ng_training <- pums_ng_final[complete.cases(pums_ng_final[,"BTUNG"]),]
ng_imputing <- pums_ng_final[is.na(pums_ng_final$"BTUNG"),]

ng_training <- ng_training[,-83:-99]
ng_imputing <- ng_imputing[,-83:-100]
ng_training[is.na(ng_training)] <- 0
ng_imputing[is.na(ng_imputing)] <- 0


All the categorical matching variables in rec and don 
 data.frames, if present are recoded into dummies


In [221]:
write.csv(ng_training, "output/ng_training.csv", row.names =FALSE)
write.csv(ng_imputing, "output/ng_imputing.csv", row.names =FALSE)

In [222]:
# statmatch other BTU
group.v <- c("x_heatfuel")
X.mtc <- c("x_yearother_std")
X.join <- c("SERIALNO","x_yearother", "x_heatfuel")

out.other<- NND.hotdeck(data.rec = recs, data.don = pums, match.vars = X.mtc, don.class = group.v, constrained = TRUE )
obtu <- create.fused(data.rec = recs, data.don = pums, mtc.ids = out.other$mtc.ids, z.vars=X.join)
obtu$dist <- out.other$dist.rd
obtu <- obtu[obtu$dist<1, ]
obtu <- obtu[,c("SERIALNO","BTUOTHER")]
pums_other_final <- merge(pums, obtu, by="SERIALNO", all.x = TRUE)

other_training <- pums_other_final[complete.cases(pums_other_final[,"BTUOTHER"]),]
other_imputing <- pums_other_final[is.na(pums_other_final$"BTUOTHER"),]
other_training <- other_training[,-83:-99]
other_imputing <- other_imputing[,-83:-100]
other_training[is.na(other_training)] <- 0
other_imputing[is.na(other_imputing)] <- 0

write.csv(other_training, "output/other_training.csv", row.names =FALSE)
write.csv(other_imputing, "output/other_imputing.csv", row.names =FALSE)




All the categorical matching variables in rec and don 
 data.frames, if present are recoded into dummies
